In [1]:

# External Dependencies
import os                          # Access system file-tree
import sys                         # Modify system parameters
from math import isclose           # isclose()
import numpy as np                 # Data structures
from optparse import OptionParser  # Get user input
import matplotlib.ticker as mticker  # Scientific notation in labels
from matplotlib.ticker import FuncFormatter

np.set_printoptions(threshold=sys.maxsize, precision=16)

In [2]:
# Internal Dependencies
internalPath = os.getcwd() + "/../python"
print(internalPath)
sys.path.insert(0, internalPath)
from plotStyling import PlotStyling  # noqa: E402
from GSDUtil import GSDUtil  # noqa: E402

/home/aglisman/Dropbox/3_Brady_Lab/Code/potential-flow/bodies-in-potential-flow/jupyter_notebooks/../python


In [3]:
# Parameters
relative_path_base = os.getcwd() + "/../data_tag/v1.0.0_output_last_frame"
relative_path = relative_path_base + "/2021-08-27.09-57-48_collinear-swimmer_varyRelDisp"
output_dir = "figures"
output_dir = relative_path + "/" + output_dir + "/"
gsd_files = []
epsOutput = True


# Correctly get scientific notation in text elements
def scientific(x, pos):
    return '%0.2e' % x


scientific_formatter = FuncFormatter(scientific)
fmt = mticker.FuncFormatter(scientific_formatter)


In [33]:
# Load data from GSD

# Uncompress data
os.system(f"\cat {relative_path}.tar.xz.parta* > {relative_path}.tar.xz" )  # combine split files
os.system(f"tar xf {relative_path}.tar.xz")  # untar data

try:
    # Loop through all subdirectories in the 'data' directory
    for root, dirs, files in os.walk(relative_path, topdown=True):
        dirs.sort()
        files.sort()

        for file in files:  # Loop through all files in given directory
            if (".gsd" in file):
                with open(root + "/" + file) as g:
                    cur_gsd = GSDUtil(g.name, create_gsd=False)
                    gsd_files.append(cur_gsd)

    assert(len(gsd_files) > 0)

except:  # No files found
    raise IOError(
        f"Failure to load data. No files found in relPath {relative_path}")


# Remove decompressed data
os.system(f"rm -rf {relative_path}.tar.xz")
os.system(f"rm -rf {relative_path}")


0

In [6]:
# Load data into np structures

CoM_disp = np.zeros(len(gsd_files), dtype=np.double)
R_avg = np.zeros_like(CoM_disp, dtype=np.double)
phaseShift = np.zeros_like(CoM_disp, dtype=np.double)
U0 = np.zeros_like(CoM_disp, dtype=np.double)
omega = np.zeros_like(CoM_disp, dtype=np.double)
epsilon = np.zeros_like(CoM_disp, dtype=np.double)
final_t = np.zeros_like(CoM_disp, dtype=np.double)

for i in range(len(gsd_files)):

    # Data from final frame
    gsd_files[i].snapshot = gsd_files[i].trajectory.read_frame(
        gsd_files[i].trajectory.file.nframes - 1)
    final_t[i] = gsd_files[i].snapshot.log['integrator/t']
    CoM_disp_comp = gsd_files[i].snapshot.log['particles/double_position'][1]

    # Data from initial frame (not 0)
    gsd_files[i].snapshot = gsd_files[i].trajectory.read_frame(1)
    R_avg[i] = gsd_files[i].snapshot.log['swimmer/R_avg']
    phaseShift[i] = gsd_files[i].snapshot.log['swimmer/phase_shift']
    U0[i] = gsd_files[i].snapshot.log['swimmer/U0']
    omega[i] = gsd_files[i].snapshot.log['swimmer/omega']
    epsilon[i] = U0[i] / R_avg[i] / omega[i]
    CoM_disp_comp -= gsd_files[i].snapshot.log['particles/double_position'][1]

    CoM_disp[i] = np.linalg.norm(CoM_disp_comp)

In [9]:
# NOTE: Assuming each GSD has same number of frames
nframes = gsd_files[0].trajectory.file.nframes

# ANCHOR: Orientation vector (not unit norm)
q = np.zeros((len(gsd_files), 3, nframes - 2), dtype=np.double)
q0 = np.zeros((len(gsd_files), 3), dtype=np.double)

# ANCHOR: temporal data
time = np.zeros((len(gsd_files), nframes - 2), dtype=np.double)

# ANCHOR: kinematic data (simulation_number, particle_number, dimension_number, frame_number)
positions = np.zeros((len(gsd_files), 3, 3, nframes - 2), dtype=np.double)
velocities = np.zeros_like(positions, dtype=np.double)
accelerations = np.zeros_like(positions, dtype=np.double)

In [10]:
for i in range(len(gsd_files)):  # loop over all GSD files

    gsd_current = gsd_files[i]
    q_current = np.zeros((3, nframes - 2), dtype=np.double)
    q_current_norm = np.zeros((nframes - 2), dtype=np.double)

    for j in range(1, nframes - 1):  # loop over all frames in GSD file

        snapshot_current = gsd_current.trajectory.read_frame(j)

        q_current[:, j - 1] = snapshot_current.log['particles/double_position'][0]
        q_current[:, j - 1] -= snapshot_current.log['particles/double_position'][2]

        time[i, j - 1] = snapshot_current.log['integrator/t']

        positions[i, :, :, j -
                  1] = snapshot_current.log['particles/double_position']
        velocities[i, :, :, j -
                   1] = snapshot_current.log['particles/double_velocity']
        accelerations[i, :, :,  -
                      1] = snapshot_current.log['particles/double_moment_inertia']

    q[i, :, :] = q_current

q0 = q[:, :, 0]

In [11]:
q0 = q[:, :, 0]
q0_norm = np.linalg.norm(q0, axis=1)
q0 = np.divide(q0, q0_norm[:, np.newaxis])

print(q0)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [12]:
# ANCHOR: Angular orientational displacement
theta = np.zeros((len(gsd_files), nframes - 2), dtype=np.double)
theta_norm = np.zeros((nframes - 2), dtype=np.double)

for i in range(len(gsd_files)):

    theta[i, :] = np.dot(q0[i, :].reshape(1, 3), q[i, :, :])

    print(np.array(theta[i, 0] - theta[i, -1], dtype=np.double))

    theta_norm = np.linalg.norm(q0[i, :]) * np.linalg.norm(q[i, :, :], axis=0)
    theta[i, :] = theta[i, :] / theta_norm

print(np.array(theta[:, 0] - theta[:, -1], dtype=np.double))

2.1280754936015e-12
1.7434942378713458e-12
2.282618538629322e-12
2.3501200985265314e-12
2.008171406941983e-12
2.213340621892712e-12
1.9593215938584763e-12
2.1174173525650986e-12
2.2319923687064147e-12
1.9744206269933784e-12
2.0907719999740948e-12
2.036593116372387e-12
2.0028423364237824e-12
2.1396218130576017e-12
1.9877433032888803e-12
2.147615418834903e-12
2.1396218130576017e-12
2.149391775674303e-12
2.0046186932631826e-12
2.0392576516314875e-12
2.099653784171096e-12
2.1040946762695967e-12
2.2852830738884222e-12
2.1174173525650986e-12
2.0952128920725954e-12
2.1440627051561023e-12
2.099653784171096e-12
2.136069099378801e-12
2.2009061240169103e-12
2.1085355683680973e-12
2.1795898419441073e-12
2.0872192862952943e-12
2.0943247136528953e-12
2.0303758674344863e-12
2.113864638886298e-12
2.1511681325137033e-12
2.1156409957256983e-12
2.05702122002549e-12
2.0339285811132868e-12
2.3447910280083306e-12
2.1529444893531036e-12
2.0250467969162855e-12
2.0747847884194925e-12
2.1600499167107046e-12
2.1

In [17]:
# relative separation between particle pairs
R_loc = positions[:, 1, :, :]
R_loc_init = positions[:, 1, :, 0]
DR_loc = R_loc - R_loc_init[:, :, np.newaxis]
R_12 = positions[:, 0, :, :] - R_loc
R_32 = positions[:, 2, :, :] - R_loc
# distance between particle pairs
Dr_Loc = np.linalg.norm(DR_loc, axis=1)
r_12 = np.linalg.norm(R_12, axis=1)
r_32 = np.linalg.norm(R_32, axis=1)

print(r_12[0, 0]-r_12[0, -1])
print(time[0, -1] - 1.0)

1.6888712650597881e-12
7.918110611626616e-12


In [19]:
# relative velocities between particle pairs
U_loc = velocities[:, 1, :, :]
U_12 = velocities[:, 0, :, :] - U_loc
U_32 = velocities[:, 2, :, :] - U_loc
# relative velocity norms between particle pairs
u_12 = np.linalg.norm(U_12, axis=0)
u_32 = np.linalg.norm(U_32, axis=0)

In [26]:
 # Average locater point velocity
U_loc_mean = np.mean(U_loc, axis=2)
U_loc_mean_x = U_loc_mean[:, 0]

print(U_loc_mean.shape)
print(U_loc_mean_x)

(53, 3)
[-2.8299801359930215e-07 -4.7687216846066626e-07 -5.3935357689096476e-07
 -6.0006771126101369e-07 -1.1224830084724665e-06 -1.3354430601570715e-06
 -1.5226556656712175e-06 -1.6878749434212104e-06 -1.8342132890691488e-06
 -1.9642671517280209e-06 -2.0802150533525125e-06 -2.8368843553759319e-07
 -2.1838946487696874e-06 -2.3604492858694775e-06 -2.5038490256681387e-06
 -2.6214021710910996e-06 -2.7185761539736367e-06 -2.7995168054305010e-06
 -2.8674061788490447e-06 -2.9247135420677537e-06 -2.9733742702476816e-06
 -3.0149192018061198e-06 -2.8437865300696377e-07 -3.0505694017404338e-06
 -3.0813064254907255e-06 -3.1079249818885369e-06 -3.1310728136941752e-06
 -3.1512811713569759e-06 -3.1689883115453250e-06 -3.2045530255079131e-06
 -3.2308346282663532e-06 -3.2505975295316670e-06 -3.2656897675632945e-06
 -2.8506866689758883e-07 -3.2773751472601301e-06 -3.2865355217545935e-06
 -3.2937974100531897e-06 -3.2996132029680430e-06 -3.3139031634676607e-06
 -3.3207348247587242e-06 -3.324319839998884

In [28]:
# Sort the average locater velocities by R_avg
idx = np.argsort(R_avg)

U_loc_mean_x_srt = U_loc_mean_x[idx]
R_avg_srt = R_avg[idx]

In [31]:
# Get average velocity when R_avg = 4.0 (what was done in internal dynamics plot)
U_loc_mean_x_desired = U_loc_mean_x_srt[R_avg_srt == 4.0]
print(f"Average velocity over period of articulation: {U_loc_mean_x_desired}")

Average velocity over period of articulation: [-3.168988311545325e-06]
